In [168]:
random_seed=7
from numpy.random import seed
seed(random_seed)
from tensorflow import set_random_seed
set_random_seed(random_seed)
import numpy as np

In [169]:
import tensorflow as tp
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
import pandas as pd
from keras import backend as K
import math
import matplotlib.pyplot as plt
import os
# from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline

In [206]:
global varss # defining all global variables here.
global count

In [275]:
#checking if GPU is used 
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8561304084881450842
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4945621811
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10575804558279978406
physical_device_desc: "device: 0, name: GeForce GTX 1060, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [133]:
def mean_imputation(dataset): 
    """"""
    imputed_dataset=dataset.fillna(dataset.mean())
    return imputed_dataset

In [161]:
#split into input (X) and output (Y)
def splitXY(dataset): 
    """
    Takes numpy array as input and converts first column into Y and rest into X
    """
    m,n=dataset.shape
    Y=dataset[:,n-1]
    X=dataset[:,0:n-1]
    return X,Y

X,Y= splitXY(dataset_np_array)

In [162]:
def r_squared(y_true, y_pred):
    
    sse=tp.reduce_sum(tp.square(tp.subtract(y_pred,y_true)))
    #using sst as y_true-y_mean 's squared mean
    y_mean= tp.reduce_mean(y_true)
    sst=tp.reduce_sum(tp.square(tp.subtract(y_true,y_mean)))
    r_square=tp.subtract(float(1),tp.divide(sse,sst))
    
    """
    m=tp.to_float(tp.size(y_true))
    y_true_sum_sq=(tp.square(tp.reduce_sum(y_true)))
    y_sq_mean=tp.divide(y_true_sum_sq,m)
    sst=tp.subtract(tp.tensordot(y_true,y_true,0),m*tp.square(tp.reduce_mean(y_true)))
    r_square=tp.subtract(float(1),tp.divide(sse,sst))
    """
    return r_square

In [163]:
def adj_r_squared(y_true, y_pred):
    sse=tp.reduce_sum(tp.square(tp.subtract(y_pred,y_true)))
    #using sst as y_true-y_mean 's squared mean
    y_mean= tp.reduce_mean(y_true)
    sst=tp.reduce_sum(tp.square(tp.subtract(y_true,y_mean)))
    r_square=tp.subtract(float(1),tp.divide(sse,sst))
    
    #n=model.count_params()#this part was working before but now is not..how to get model here though?
    #n=sum([np.prod(K.get_value(w).shape) for w in model.trainable_weights])
    n=tp.to_float(count)
    #m=tp.to_float(tp.size(y_true))
    rdf=tp.divide(tp.subtract(tp.to_float(instances),float(1)),tp.subtract(tp.to_float(instances),n))
    r_adj_square=tp.subtract(float(1),tp.tensordot(rdf,tp.subtract(float(1),r_square),0))
    return r_adj_square

In [212]:
def perceptronModel(X_dim=7,optimizer_name="adam"):
    """Takes number of X features and activation name as input and outputs a keras model for perceptron"""
    model = Sequential()
    model.add(Dense(1, input_dim=X_dim,activation="linear",kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer='zeros'))
    #model.add(Dense(1,activation='linear',kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    #model.add(Dense(1,kernel_initializer='normal'))
    global count
    count=model.count_params()
    #model.add(Dense(1, input_dim=X_dim, activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    # Compile model
    sgd=optimizers.SGD(lr=0.05, momentum=0.05, decay=0.0, nesterov=False)
    rmsprop=keras.optimizers.RMSprop(lr=0.05, rho=0.9, epsilon=None, decay=0.0)
    model.compile(loss='mean_squared_error',optimizer=optimizer_name,metrics=['mse',r_squared,adj_r_squared])
    
    return model

In [277]:
def neuralNetwork3LModel(X_dim,activation_name='relu',optimizer_name="adam"):
    """Takes number of X features and activation name as input and outputs a keras model for perceptron"""
    model = Sequential()
    model.add(Dense(7, input_dim=X_dim,activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer='zeros'))
    model.add(Dense(1,activation='linear',kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer='zeros'))
    #model.add(Dense(1,kernel_initializer='normal'))
    
    #model.add(Dense(1, input_dim=X_dim, activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    # Compile model
    global count
    count=model.count_params()#-X_dim+1
    sgd=optimizers.SGD(lr=0.05, momentum=0.1, decay=0.0, nesterov=False)
    rmsprop=keras.optimizers.RMSprop(lr=0.05, rho=0.1, epsilon=None, decay=0.0)
    model.compile(loss='mean_squared_error',optimizer=optimizer_name,metrics=['mse',r_squared,adj_r_squared])

    
    return model

In [278]:
def neuralNetwork5LModel(X_dim,activation_name='relu',optimizer_name="adam"):
    """Takes number of X features and activation name as input and outputs a keras model for perceptron"""
    model = Sequential()
    model.add(Dense(5, input_dim=X_dim,activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer='zeros'))
    model.add(Dense(5,activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer='zeros'))
    model.add(Dense(5,activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer='zeros'))
    model.add(Dense(1,activation='linear',kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer='zeros'))
    #model.add(Dense(1,kernel_initializer='normal'))
    
    #model.add(Dense(1, input_dim=X_dim, activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    # Compile model
    count=model.count_params() #-3*X_dim +1
    print('Count of parameters :'+str(count))
    sgd=optimizers.SGD(lr=0.05, momentum=0.05, decay=0.0, nesterov=False)
    model.compile(loss='mean_squared_error',optimizer=optimizer_name,metrics=['mse',r_squared,adj_r_squared])

    
    return model

In [279]:
def fit_model(X,Y,name='NeuralNetwork3L',activation_name="relu",optimizer_name="adam"):
    if(len(X.shape))==1: 
        X_size=X.shape
        X_dim=1
    else :    
        (X_size,X_dim)=X.shape
    global instances
    instances=X_size
    if(name=='Perceptron'):
        model =perceptronModel(X_dim,optimizer_name)
    elif(name=='NeuralNetwork3L'):
        model =neuralNetwork3LModel(X_dim,activation_name,optimizer_name)
    elif(name=='NeuralNetwork5L'):
        model =neuralNetwork5LModel(X_dim,activation_name,optimizer_name)
    else: 
        raise Exception("Model Name is not correct: Please choose between given models")
    
    count=model.count_params() #-3*X_dim +1
    print('Count of parameters :'+str(count))
    model.summary()
    #early stopping 
    es = EarlyStopping(monitor='mean_squared_error', mode='min', verbose=1, patience=10)
    # Fit the model
    with tp.device('/device:GPU:0'):
        model.fit(X, Y, epochs=4000, batch_size=50,callbacks=[es])
        
    return model
#model=fit_model(X,Y)

In [270]:
def eval_model(X,Y,model):
#model evaluation.

    scores = model.evaluate(X, Y, verbose=0)
        
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]))
    print("%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
    print("%s: %.2f%%" % (model.metrics_names[3], scores[3]*100))
    return scores
#eval_model(X,Y,model)

In [271]:
#To find RCV value
def cross_val_split(dataset):
    #dataset is split into 80:20 ratio of train and test
    np.random.shuffle(dataset)
    m,n=dataset_np_array.shape
    count=math.ceil(m*0.8)
    training, test = dataset[:count,:], dataset[count:,:]
    X_train,Y_train=splitXY(training)
    X_test,Y_test=splitXY(test)
#     model=fit_model(X_train,Y_train)
#     # evaluate the model
#     scores = eval_model(X_test,Y_test,model)
#     r_square_cv=dict_scores[model.metrics_names[2]]=scores[2]
    return X_train,Y_train,X_test,Y_test   
#cross_val_split(dataset_np_array)

In [272]:
def forward_selection(dataset,modelname="Perceptron",activation_name="relu",optimizer_name="adam"): 
    X,Y=splitXY(dataset)
    X_train,Y_train,X_test,Y_test=cross_val_split(dataset)    
    m,n=X.shape
    X_cols = [] # buffer list that tells which column index is in X right now...
    X_cols_cv=[]
    X_set=np.empty((m,0))#for test and train with cv..this would need to be different..
    #need to save the below values for all models calculated in for loop and only save the best sse's
    
    m_train,n_train=X_train.shape
    m_test,n_test=X_test.shape
    X_train_set=np.empty((m_train,0))
    X_test_set=np.empty((m_test,0))
    
    r_square=[]
    r_adj=[]
    r_square_cv=[]
    r_adj_cv=[]
    
    while(len(X_cols)<n):
        sse=[]
        sse_cv=[]
        r_square_best=[]
        r_adj_best=[]
        r_square_cv_best=[]
        r_adj_cv_best=[]
        for i in range(0,n) :
            if i not in X_cols :
                #for rsq and radjsq
                mno,nno=X_train_set.shape
                X_train_set_copy=X_train[:,X_cols+[i]]
                model=fit_model(X_train_set_copy,Y_train,modelname,activation_name,optimizer_name)
            
                X_test_set_copy=X_test[:,X_cols+[i]]
                scores=eval_model(X_test_set_copy,Y_test,model)
                r_square_cv_best.append(scores[2])
                r_adj_cv_best.append(scores[3])
                
                scores=eval_model(X_train_set_copy,Y_train,model)
                sse.append(float(scores[1]))
                r_square_best.append(scores[2])
                r_adj_best.append(scores[3])
                
                
            else: 
                sse.append(math.inf)#to
                r_square_best.append(float('-inf'))
                r_adj_best.append(float('-inf'))                
                r_square_cv_best.append(float('-inf'))
                r_adj_cv_best.append(float('-inf'))
                
        #for rsq and radjsq
        best_index=sse.index(min(sse))
        X_cols.append(best_index)
        r_square_cv.append(r_square_cv_best[best_index])
        r_adj_cv.append(r_adj_cv_best[best_index])
        r_square.append(r_square_best[best_index])
        r_adj.append(r_adj_best[best_index])

    return r_square,r_adj,r_square_cv


In [273]:
def save_plots(datasetno,r_square,r_adj,r_square_cv,modelname):
    m,n=dataset.shape
    t=[i+1 for i in range(n-1)]
    plt.plot(t,[value*100 for value in r_square], 'r--', linewidth=2.0,label="R-squared")
    plt.plot( t, [value*100 for value in r_adj], 'bs--',linewidth=2.0,label="R^2-adjusted")
    plt.plot( t,[value*100 for value in r_square_cv],'g^--',linewidth=2.0,label="Rcv-square")
    plt.xlabel("n:Forward Selection")
    plt.ylabel("R square values")
    plt.legend()
    x=str(datasetno)
    if not os.path.exists('../../plots/'+x):
        os.makedirs('../../plots/'+x)
    plt.savefig('../../plots/'+x+'/'+modelname+'.png')
    plt.show()

In [ ]:
#experiment manual script
dataset = pd.read_csv("../../data/1.csv", delimiter=",")
dataset=mean_imputation(dataset)
dataset_np_array=dataset.values
X,Y=splitXY(dataset_np_array)
model=fit_model(X,Y)

In [ ]:
#running script
activations_exp=['sigmoid','relu','linear','tanh']
optimizer_exp=['sgd','adam','rmsprop','adagrad']
modelnames=["NeuralNetwork3L","NeuralNetwork5L"]


for datasetno in range(9,-1,-1): 
    for modelname in modelnames: 
        datasetname=datasetno+1
        dataset = pd.read_csv("../../data/"+str(datasetname)+".csv", delimiter=",")
        dataset=mean_imputation(dataset)
        dataset_np_array=dataset.values
        r_square,r_adj,r_square_cv=forward_selection(dataset_np_array,modelname,"relu","adam")
        save_plots(datasetno+1,r_square,r_adj,r_square_cv,modelname)

Count of parameters :22
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_438 (Dense)            (None, 7)                 14        
_________________________________________________________________
dense_439 (Dense)            (None, 1)                 8         
Total params: 22
Trainable params: 22
Non-trainable params: 0
_________________________________________________________________
Epoch 1/4000
247/247 [==============================] - 3s 14ms/sample - loss: 351.9777 - mean_squared_error: 351.9778 - r_squared: -0.5392 - adj_r_squared: -0.6828
Epoch 2/4000
247/247 [==============================] - 0s 208us/sample - loss: 350.8668 - mean_squared_error: 350.8669 - r_squared: -0.5105 - adj_r_squared: -0.6515
Epoch 3/4000
247/247 [==============================] - 0s 204us/sample - loss: 349.6849 - mean_squared_error: 349.6849 - r_squared: -0.5065 - adj_r_squared: -0.6472
Epoch 4/4000
247/247 

247/247 [==============================] - 0s 234us/sample - loss: 314.0112 - mean_squared_error: 314.0112 - r_squared: -0.3401 - adj_r_squared: -0.4652
Epoch 47/4000
247/247 [==============================] - 0s 215us/sample - loss: 313.4025 - mean_squared_error: 313.4025 - r_squared: -0.3453 - adj_r_squared: -0.4708
Epoch 48/4000
247/247 [==============================] - 0s 275us/sample - loss: 312.6688 - mean_squared_error: 312.6688 - r_squared: -0.3390 - adj_r_squared: -0.4640
Epoch 49/4000
247/247 [==============================] - 0s 230us/sample - loss: 311.9821 - mean_squared_error: 311.9821 - r_squared: -0.3240 - adj_r_squared: -0.4476
Epoch 50/4000
247/247 [==============================] - 0s 260us/sample - loss: 311.3197 - mean_squared_error: 311.3196 - r_squared: -0.3328 - adj_r_squared: -0.4572
Epoch 51/4000
247/247 [==============================] - 0s 218us/sample - loss: 310.6679 - mean_squared_error: 310.6679 - r_squared: -0.3212 - adj_r_squared: -0.4445
Epoch 52/400

247/247 [==============================] - 0s 224us/sample - loss: 286.2731 - mean_squared_error: 286.2732 - r_squared: -0.2257 - adj_r_squared: -0.3400
Epoch 96/4000
247/247 [==============================] - 0s 187us/sample - loss: 285.7650 - mean_squared_error: 285.7650 - r_squared: -0.2196 - adj_r_squared: -0.3334
Epoch 97/4000
247/247 [==============================] - 0s 239us/sample - loss: 285.3367 - mean_squared_error: 285.3367 - r_squared: -0.2122 - adj_r_squared: -0.3254
Epoch 98/4000
247/247 [==============================] - 0s 238us/sample - loss: 284.8830 - mean_squared_error: 284.8830 - r_squared: -0.2097 - adj_r_squared: -0.3226
Epoch 99/4000
247/247 [==============================] - 0s 207us/sample - loss: 284.4250 - mean_squared_error: 284.4250 - r_squared: -0.2104 - adj_r_squared: -0.3233
Epoch 100/4000
247/247 [==============================] - 0s 238us/sample - loss: 284.0084 - mean_squared_error: 284.0085 - r_squared: -0.2048 - adj_r_squared: -0.3172
Epoch 101/4